## Import Libraries and Declare API Key

In [2]:
import requests
import pandas as pd
import numpy as np
import signal
import time
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
API_KEY = {'X-API-key': 'RPCJTKLW'}

## Accesing the Past Heat History

### Historical Data Retrieval

In [4]:
pd_new = ''
with requests.Session() as s:
    s.headers.update(API_KEY)
    resp = s.get('http://localhost:9999/v1/news')
    if resp.ok:
        price_hist = resp.json()
        pd_list = pd.read_html(price_hist[0]['body'])
        pd_new = pd_list[0]
        pd_new = pd_new.iloc[: , 1:]
        pd_new.rename(columns=pd_new.iloc[0], inplace = True)
        pd_new.drop([0], inplace = True)
        pd_new.index = np.arange(0, len(pd_new))
        pd_new = pd_new.apply(pd.to_numeric, errors='ignore')
    else:
        print('API Error')

pd_new

,RSM1000,NGN,WHEL,GEAR
0,96.65,25.17,26.35,24.59
1,96.24,24.98,26.35,24.52
2,96.50,25.09,26.34,24.61
3,96.22,25.03,26.36,24.48
4,97.07,25.34,26.24,24.59
...,...,...,...,...
296,99.98,24.99,25.02,25.14
297,99.88,24.89,25.10,25.10
298,99.70,24.80,25.12,25.10
299,99.85,24.93,25.04,24.97


### Observed Historical Correlation

In [ ]:
corr_df = pd_new.corr()
#reset symbol as index (rather than 0-X)
corr_df.head(10)
corr_df.style.set_caption("Observed Historical Correlation")

### Observed Historical Volatillity

In [ ]:
def volatility(tick):
    pd_new['Log returns'] = np.log(pd_new[tick]/pd_new[tick].shift())
    volatility = pd_new['Log returns'].std()*1200**.5
    return volatility

In [ ]:
lst = [[volatility('RSM1000'), volatility('NGN'), volatility('WHEL'), volatility('GEAR')]]
df_vol = pd.DataFrame(lst, columns =['RSM1000', 'GEAR', 'WHEL', 'NGN'])
df_vol.style.set_caption("Observed Historical Volatillity")

### Observed Historical Beta

In [ ]:
def market_beta(tick):
    pd_new['Log returns_RSM'] = np.log(pd_new['RSM1000']/pd_new['RSM1000'].shift())
    pd_new['Log returns'] = np.log(pd_new[tick]/pd_new[tick].shift())
    
    beta = corr_df['RSM1000'][tick] * (pd_new['Log returns'].std() / pd_new['Log returns_RSM'].std())
    return beta  

In [ ]:
lst = [[market_beta('GEAR'), market_beta('WHEL'), market_beta('NGN')]]
df_bet = pd.DataFrame(lst, columns =['GEAR', 'WHEL', 'NGN'])
df_bet.style.set_caption("Observed Historical Beta")

## Current Heat Trading Algorithm

### Basic Buy and Sell Functions

In [ ]:
shutdown = False

class ApiException(Exception):
    pass

def signal_handler(signum, frame):
    global shutdown
    signal.signal(signal.SIGINT, signal.SIG_DFL)
    shutdown = True

def get_tick(session):
    resp = session.get('http://localhost:9999/v1/case')
    if resp.ok:
        case = resp.json()
        return case['tick']
    raise ApiException('Authorization error. Please check API key.')

def ticker_bid_ask(session, ticker):
    payload = {'ticker': ticker}
    resp = session.get('http://localhost:9999/v1/securities/book', params = payload)
    if resp.ok:
        book = resp.json()
        return book['bids'][0]['price'], book['asks'][0]['price']
    raise ApiException('Authorization error. Please check API key.')

def buy(session, ticker, type, q, price = None):
    if type == 'MARKET':
        resp = session.post('http://localhost:9999/v1/orders', params = {'ticker': ticker, 'type': 'MARKET', 'quantity': q, 'action': 'BUY'})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')
    else:
        resp = session.post('http://localhost:9999/v1/orders',
                            params={'ticker': ticker, 'type': 'LIMIT', 'quantity': q, 'action': 'BUY', 'price': price})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')

def sell(session, ticker, type, q, price = None):
    if type == 'MARKET':
        resp = session.post('http://localhost:9999/v1/orders', params = {'ticker': ticker, 'type': 'MARKET', 'quantity': q, 'action': 'SELL'})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')
    else:
        resp = session.post('http://localhost:9999/v1/orders',
                            params={'ticker': ticker, 'type': 'LIMIT', 'quantity': q, 'action': 'SELL', 'price': price})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')

def cancel(session, order_id):
    order_id = order_id # assuming the order to cancel has ID 100
    resp = session.delete('http://localhost:9999/v1/orders/{}'.format(order_id))
    if resp.ok:
        status = resp.json()
        success = status['success']
        print('The order was successfully cancelled?', success)
        
def get_limit(session):
    resp = session.get('http://localhost:9999/v1/limits')
    if resp.ok:
        lim = resp.json()[0]
        return lim['gross'], lim['net'], lim['gross_limit'], lim['net_limit']
    else:
        print('API Error!')
        

### Divergence Functions

In [ ]:
def init_price(session, ticker):
    resp = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'NGN'})
    if resp.ok:
        book = resp.json()
        return book[-1]['close']

def cur_price(session, ticker):
    resp = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'NGN'})
    if resp.ok:
        book = resp.json()
        return book[0]['close']
    
def ptd_ret(session, ticker):
    return cur_price(session, ticker)/init_price(session, ticker) - 1

def e_ptd_ret(session, ticker):
    return ptd_ret(session, 'RSM1000') * df_bet[ticker][0]

def div(session, ticker):
    return ptd_ret(session, ticker) - e_ptd_ret(session, ticker)

## Algorithm Execution Section

In [ ]:
with requests.Session() as s:
    s.headers.update(API_KEY)
    gross, net, gross_limit, net_limit = get_limit(s)
    print('gross: ', gross)
    print('net: ', net)
    print('gross_limit: ', gross_limit)
    print('net_limit: ', net_limit)

In [ ]:
signal.signal(signal.SIGINT, signal_handler)

with requests.Session() as s:
    s.headers.update(API_KEY)
    while True:
        tick = get_tick(s)
        price = 10000
        if div(s, 'NGN') > 0:
            buy(s, 'NGN','MARKET',  -price)
        if div(s, 'WHEL') > 0:
            buy(s, 'WHEL','MARKET', -price)
        if div(s, 'GEAR') > 0:
            buy(s, 'GEAR','MARKET', -price)
        if div(s, 'NGN') < 0:
            buy(s, 'NGN','MARKET', price)
        if div(s, 'WHEL') < 0:
            buy(s, 'WHEL','MARKET', price)
        if div(s, 'GEAR') < 0:
            buy(s, 'GEAR','MARKET', price)
        time.sleep(1)